<a href="https://colab.research.google.com/github/pierrelouisbescond/medium_articles/blob/main/medium_christmas_run_with_OR_Tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### *PEP8 CHECKED*

## Notebook Preparation

In [ ]:
!pip install ortools
!pip install geopandas

In [ ]:
import pandas as pd
import numpy as np

import geopandas as gpd

from scipy.spatial import distance_matrix

import plotly.graph_objects as go

# Map Creation Librairies
import folium

# OR-Tools Solvers
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2


## Macro-level (cities) optimization

### Data Preparation

In [ ]:
ww_cities = pd.read_excel("/content/drive/My Drive/Medium/worldcities.xlsx", index_col=1)
ww_cities.sample(10)


,city,lat,lng,country,iso2,iso3,admin_name,capital,population,id
city_ascii,,,,,,,,,,
Prijedor,Prijedor,44.9667,16.7000,Bosnia And Herzegovina,BA,BIH,"Srpska, Republika",minor,89397.0,1070565353
Woodlesford,Woodlesford,53.7567,-1.4530,United Kingdom,GB,GBR,Leeds,NaN,21010.0,1826564728
Larisa,Lárisa,39.6385,22.4131,Greece,GR,GRC,Thessalía,admin,144651.0,1300003141
Barehra,Barehra,27.7500,78.5667,India,IN,IND,Uttar Pradesh,NaN,19542.0,1356242932
Kranuan,Kranuan,16.7081,103.0811,Thailand,TH,THA,Khon Kaen,minor,10726.0,1764256426
Kurikka,Kurikka,62.6167,22.4000,Finland,FI,FIN,Etelä-Pohjanmaa,minor,21734.0,1246688374
Pomaz,Pomáz,47.6472,19.0269,Hungary,HU,HUN,Pest,NaN,17139.0,1348490155
Cajamarca,Cajamarca,-7.1644,-78.5106,Peru,PE,PER,Cajamarca,admin,201329.0,1604091119
Ahlen,Ahlen,51.7633,7.8911,Germany,DE,DEU,North Rhine-Westphalia,NaN,52582.0,1276843072


In [ ]:
ww_cities.shape

(26562, 10)

In [ ]:
ww_cities = ww_cities[(["city", "lat", "lng", "capital", "population"])]
ww_cities = ww_cities[(ww_cities["capital"] == "primary") & (ww_cities["population"] >= 1000000)]
ww_cities.sample(10, random_state=2)


,city,lat,lng,capital,population
city_ascii,,,,,
Montevideo,Montevideo,-34.8667,-56.1667,primary,1319108.0
Abidjan,Abidjan,5.3364,-4.0267,primary,4980000.0
Caracas,Caracas,10.5000,-66.9333,primary,1943901.0
Baghdad,Baghdad,33.3500,44.4167,primary,5796000.0
Bogota,Bogotá,4.6126,-74.0705,primary,9464000.0
Dar es Salaam,Dar es Salaam,-6.8000,39.2833,primary,6698000.0
Manila,Manila,14.5958,120.9772,primary,23088000.0
Ankara,Ankara,39.9300,32.8500,primary,5503985.0
Rangoon,Rangoon,16.8000,96.1500,primary,5430000.0


### World Map Creation

In [ ]:
# Let's initialize our World map with Folium
ww_map_geopd = folium.Map(location=[20, 0],
                    tiles="Stamen Terrain",
                    zoom_start=2,
                    width="80%",
                    height="80%")

# For an easier visualization, we keep on 15 capitals
ww_cities_sample = ww_cities.sample(15, random_state=2)

# We also need to include Sant Claus village, located in Rovaniemi (Finland)
ww_cities_sample.loc["Pole_North", "city"] = "Pole North"
ww_cities_sample.loc["Pole_North", "lng"] = 27.72
ww_cities_sample.loc["Pole_North", "lat"] = 66.50

# We can convert the original DataFrame to a GeoPandas DataFrame to ease
# the map creation
ww_cities_sample_geopd = gpd.GeoDataFrame(ww_cities_sample, geometry=gpd.points_from_xy(ww_cities_sample.lng, ww_cities_sample.lat), crs="EPSG:4326")

# Let's add a marker for each of the 15 capitals
folium.GeoJson(ww_cities_sample_geopd).add_to(ww_map_geopd)

# And display the Wolrd Map
ww_map_geopd


### Distance Matrix Calculation

In [ ]:
# Only capitals' name, longitude and latitude are needed
ww_cities_sample_coordinates = ww_cities_sample_geopd[["lng", "lat"]]

# Let's calculate the distance matrix
ww_cities_sample_dist_mtx = pd.DataFrame(distance_matrix(ww_cities_sample_coordinates.values, ww_cities_sample_coordinates.values),
                                   index=ww_cities_sample_coordinates.index,
                                   columns=ww_cities_sample_coordinates.index
                                         )
# Conversion of the DataFrame to a NumPy array
ww_cities_sample_dist_mtx_np = ww_cities_sample_dist_mtx.to_numpy()

# Let's display the corresponding result
round(ww_cities_sample_dist_mtx)


city_ascii,Montevideo,Abidjan,Caracas,Baghdad,Bogota,Dar es Salaam,Manila,Ankara,Rangoon,Paris,Vienna,Nouakchott,Prague,London,Tokyo,Pole_North
city_ascii,,,,,,,,,,,,,,,,
Montevideo,0.0,66.0,47.0,122.0,43.0,99.0,184.0,116.0,161.0,102.0,110.0,66.0,110.0,103.0,208.0,132.0
Abidjan,66.0,0.0,63.0,56.0,70.0,45.0,125.0,51.0,101.0,44.0,47.0,17.0,48.0,46.0,147.0,69.0
Caracas,47.0,63.0,0.0,114.0,9.0,108.0,188.0,104.0,163.0,79.0,91.0,52.0,90.0,78.0,208.0,110.0
Baghdad,122.0,56.0,114.0,0.0,122.0,40.0,79.0,13.0,54.0,45.0,32.0,62.0,34.0,48.0,95.0,37.0
Bogota,43.0,70.0,9.0,122.0,0.0,114.0,195.0,113.0,171.0,88.0,100.0,60.0,99.0,88.0,216.0,119.0
Dar es Salaam,99.0,45.0,108.0,40.0,114.0,0.0,84.0,47.0,62.0,67.0,60.0,61.0,62.0,70.0,109.0,74.0
Manila,184.0,125.0,188.0,79.0,195.0,84.0,0.0,92.0,25.0,123.0,110.0,137.0,112.0,127.0,28.0,107.0
Ankara,116.0,51.0,104.0,13.0,113.0,47.0,92.0,0.0,67.0,32.0,18.0,53.0,21.0,35.0,107.0,27.0
Rangoon,161.0,101.0,163.0,54.0,171.0,62.0,25.0,67.0,0.0,99.0,86.0,112.0,88.0,102.0,47.0,85.0


### Routing Solver

In [ ]:
%%time

# Data Model Creation function
def create_data_model(distance_matrix, north_pole_index):

    data = {}
    data['distance_matrix'] = distance_matrix
    data['num_vehicles'] = 1
    data['depot'] = north_pole_index # Where we are starting from
    return data

# We need to know the index number of the North Pole
north_pole_index = ww_cities_sample_dist_mtx.index.get_loc("Pole_North")

# Let's create the Data Model with the distance matrix and the North Pole Index
data = create_data_model(ww_cities_sample_dist_mtx_np, north_pole_index)

# Routing Model Initialization
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_vehicles'],
                                       data['depot'])

routing = pywrapcp.RoutingModel(manager)

# Distance matrix callback function
def distance_callback(from_index, to_index):

    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)

routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Search parameters definition
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
search_parameters.local_search_metaheuristic = (
    routing_enums_pb2.LocalSearchMetaheuristic.AUTOMATIC)

def print_solution(manager, routing, solution):

    route_array = []
    index = routing.Start(0)

    route_distance = 1
    while not routing.IsEnd(index):
        route_array.append(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)

    return route_array

# Solver launch
solution = routing.SolveWithParameters(search_parameters)

if solution:
    route_array = print_solution(manager, routing, solution)

print(route_array)


[15, 10, 12, 9, 13, 11, 2, 4, 0, 1, 5, 8, 6, 14, 3, 7]
CPU times: user 14.7 ms, sys: 2.06 ms, total: 16.8 ms
Wall time: 17.1 ms


### Route solution and display

In [ ]:
# Let's sort the capital according to the route defined
ww_cities_sample_coordinates_sorted = ww_cities_sample_coordinates.reset_index().copy()
ww_cities_sample_coordinates_sorted["rank"] = np.zeros(len(ww_cities_sample_coordinates_sorted))

for index in ww_cities_sample_coordinates_sorted.index:
  ww_cities_sample_coordinates_sorted.loc[index, "rank"] = route_array.index(index)

ww_cities_sample_coordinates_sorted = ww_cities_sample_coordinates_sorted.sort_values(by="rank")
print(ww_cities_sample_coordinates_sorted)


       city_ascii       lng      lat  rank
15     Pole_North   27.7200  66.5000   0.0
10         Vienna   16.3731  48.2083   1.0
12         Prague   14.4167  50.0833   2.0
9           Paris    2.3522  48.8566   3.0
13         London   -0.1275  51.5072   4.0
11     Nouakchott  -15.9744  18.0783   5.0
2         Caracas  -66.9333  10.5000   6.0
4          Bogota  -74.0705   4.6126   7.0
0      Montevideo  -56.1667 -34.8667   8.0
1         Abidjan   -4.0267   5.3364   9.0
5   Dar es Salaam   39.2833  -6.8000  10.0
8         Rangoon   96.1500  16.8000  11.0
6          Manila  120.9772  14.5958  12.0
14          Tokyo  139.6922  35.6897  13.0
3         Baghdad   44.4167  33.3500  14.0
7          Ankara   32.8500  39.9300  15.0


In [ ]:
# Let's add this journey to the World Map (credit to https://github.com/matthiashhh)
folium.PolyLine(ww_cities_sample_coordinates_sorted[["lat", "lng"]].to_numpy(), color='blue').add_to(ww_map_geopd)
ww_map_geopd

## Micro-level (neighborhood) optimization

### Data Preparation

In [ ]:
street_names = ["Toys Avenue", "The Elves Boulevard", "Sleigh Ride", "Rudolph Alley", "Candy Street", "Reindeer Avenue", "The Artic Impasse", "Snowman Street", "Aurora Alley", "Gingerbread street"]

street_coordinates = {
'Start_lat':  ['50.674380', '50.676395', '50.676882', '50.67703', '50.676882', '50.67571', '50.67504', '50.67438', '50.67571', '50.67504'],
'Start_long': ['3.096106', '3.093955', '3.094978', '3.097701', '3.094978', '3.096349', '3.097100', '3.096121', '3.096349', '3.097100'],
'End_lat': ['50.676395', '50.676882', '50.67703', '50.67602', '50.674700', '50.67504', '50.67470', '50.67470', '50.67662', '50.676312'],
'End_long': ['3.093955', '3.094978', '3.097701', '3.099203', '3.097481', '3.097100', '3.097481', '3.097481', '3.098355', '3.099718']
}

neighboordhood = pd.DataFrame(street_coordinates, index=street_names, columns=["Start_lat", "Start_long", "End_lat", "End_long"]).astype(float)
neighboordhood


,Start_lat,Start_long,End_lat,End_long
Toys Avenue,50.674380,3.096106,50.676395,3.093955
The Elves Boulevard,50.676395,3.093955,50.676882,3.094978
Sleigh Ride,50.676882,3.094978,50.677030,3.097701
Rudolph Alley,50.677030,3.097701,50.676020,3.099203
Candy Street,50.676882,3.094978,50.674700,3.097481
Reindeer Avenue,50.675710,3.096349,50.675040,3.097100
The Artic Impasse,50.675040,3.097100,50.674700,3.097481
Snowman Street,50.674380,3.096121,50.674700,3.097481
Aurora Alley,50.675710,3.096349,50.676620,3.098355
Gingerbread street,50.675040,3.097100,50.676312,3.099718


In [ ]:
# Let's initialize our neighborhood map with Folium
neighboordhood_map = folium.Map(location=[50.675700, 3.096281], #Map centered on this position
                    # Custom tiles details:
                    tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
                    attr="Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community",
                    zoom_start=17,
                    width="80%",
                    height="80%",
                    )

for i in range(0, len(neighboordhood)):
  folium.Marker([neighboordhood.iloc[i]['Start_lat'], neighboordhood.iloc[i]['Start_long']]).add_to(neighboordhood_map)
  folium.Marker([neighboordhood.iloc[i]['End_lat'], neighboordhood.iloc[i]['End_long']]).add_to(neighboordhood_map)

# And display the neighborhood map
neighboordhood_map


### Distance Matrix Calculation

In [ ]:
# Let's calculate the difference (longitude and latitude) between every street
# start and endpoint:
neighboordhood["Delta_long"] = neighboordhood["End_long"] - neighboordhood["Start_long"]
neighboordhood["Delta_lat"] = neighboordhood["End_lat"] - neighboordhood["Start_lat"]
neighboordhood


,Start_lat,Start_long,End_lat,End_long,Delta_long,Delta_lat
Toys Avenue,50.674380,3.096106,50.676395,3.093955,-0.002151,0.002015
The Elves Boulevard,50.676395,3.093955,50.676882,3.094978,0.001023,0.000487
Sleigh Ride,50.676882,3.094978,50.677030,3.097701,0.002723,0.000148
Rudolph Alley,50.677030,3.097701,50.676020,3.099203,0.001502,-0.001010
Candy Street,50.676882,3.094978,50.674700,3.097481,0.002503,-0.002182
Reindeer Avenue,50.675710,3.096349,50.675040,3.097100,0.000751,-0.000670
The Artic Impasse,50.675040,3.097100,50.674700,3.097481,0.000381,-0.000340
Snowman Street,50.674380,3.096121,50.674700,3.097481,0.001360,0.000320
Aurora Alley,50.675710,3.096349,50.676620,3.098355,0.002006,0.000910
Gingerbread street,50.675040,3.097100,50.676312,3.099718,0.002618,0.001272


In [ ]:
# Number of intermediary locations
nb_steps = 30

neighboordhood_augmented_coordinates = pd.DataFrame(np.zeros((nb_steps * len(neighboordhood.index), 3)), columns=["Street Name", "Long", "Lat"])

counter = 0

for street_name in neighboordhood.index:

  # We define the starting point of the street
  Local_Start_long = neighboordhood.loc[street_name, "Start_long"]
  Local_Start_lat = neighboordhood.loc[street_name, "Start_lat"]

  # We calculate the longitude and latitude steps to get from the start to the end
  Long_step = neighboordhood.loc[street_name, "Delta_long"] / (nb_steps - 1)
  Lat_step = neighboordhood.loc[street_name, "Delta_lat"] / (nb_steps - 1)

  # We add the corresponding locations to the dataframe
  for step in range(nb_steps):
    neighboordhood_augmented_coordinates.loc[counter, "Street Name"] = street_name+"_"+str(step)
    neighboordhood_augmented_coordinates.loc[counter, "Long"] = Local_Start_long + step * (Long_step)
    neighboordhood_augmented_coordinates.loc[counter, "Lat"] = Local_Start_lat + step * (Lat_step)

    counter += 1

  #print(street_name,"-> processed!")

neighboordhood_augmented_coordinates = neighboordhood_augmented_coordinates.set_index("Street Name")

# We remove duplicates locations (start and end points from connected streets)
neighboordhood_augmented_coordinates = neighboordhood_augmented_coordinates.drop_duplicates()


# We calculate the distance matrix and convert it to a NumPy array
# We multiply the results by a 100.000 ratio as solver will round the values
# to preserve accuarcy
neighboordhood_mtx = pd.DataFrame(distance_matrix(neighboordhood_augmented_coordinates.values, neighboordhood_augmented_coordinates.values),
                                   index=neighboordhood_augmented_coordinates.index,
                                   columns=neighboordhood_augmented_coordinates.index) * 100000

neighboordhood_mtx = round(neighboordhood_mtx)
display(neighboordhood_mtx)

neighboordhood_mtx = neighboordhood_mtx.to_numpy()


Street Name,Toys Avenue_0,Toys Avenue_1,Toys Avenue_2,Toys Avenue_3,Toys Avenue_4,Toys Avenue_5,Toys Avenue_6,Toys Avenue_7,Toys Avenue_8,Toys Avenue_9,Toys Avenue_10,Toys Avenue_11,Toys Avenue_12,Toys Avenue_13,Toys Avenue_14,Toys Avenue_15,Toys Avenue_16,Toys Avenue_17,Toys Avenue_18,Toys Avenue_19,Toys Avenue_20,Toys Avenue_21,Toys Avenue_22,Toys Avenue_23,Toys Avenue_24,Toys Avenue_25,Toys Avenue_26,Toys Avenue_27,Toys Avenue_28,Toys Avenue_29,The Elves Boulevard_1,The Elves Boulevard_2,The Elves Boulevard_3,The Elves Boulevard_4,The Elves Boulevard_5,The Elves Boulevard_6,The Elves Boulevard_7,The Elves Boulevard_8,The Elves Boulevard_9,The Elves Boulevard_10,...,Aurora Alley_19,Aurora Alley_20,Aurora Alley_21,Aurora Alley_22,Aurora Alley_23,Aurora Alley_24,Aurora Alley_25,Aurora Alley_26,Aurora Alley_27,Aurora Alley_28,Aurora Alley_29,Gingerbread street_1,Gingerbread street_2,Gingerbread street_3,Gingerbread street_4,Gingerbread street_5,Gingerbread street_6,Gingerbread street_7,Gingerbread street_8,Gingerbread street_9,Gingerbread street_10,Gingerbread street_11,Gingerbread street_12,Gingerbread street_13,Gingerbread street_14,Gingerbread street_15,Gingerbread street_16,Gingerbread street_17,Gingerbread street_18,Gingerbread street_19,Gingerbread street_20,Gingerbread street_21,Gingerbread street_22,Gingerbread street_23,Gingerbread street_24,Gingerbread street_25,Gingerbread street_26,Gingerbread street_27,Gingerbread street_28,Gingerbread street_29
Street Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Toys Avenue_0,0.0,10.0,20.0,30.0,41.0,51.0,61.0,71.0,81.0,91.0,102.0,112.0,122.0,132.0,142.0,152.0,163.0,173.0,183.0,193.0,203.0,213.0,224.0,234.0,244.0,254.0,264.0,274.0,285.0,295.0,293.0,292.0,291.0,289.0,288.0,287.0,286.0,285.0,284.0,283.0,...,248.0,255.0,261.0,268.0,275.0,282.0,289.0,296.0,303.0,310.0,317.0,129.0,139.0,149.0,159.0,169.0,179.0,189.0,199.0,209.0,219.0,229.0,239.0,249.0,259.0,269.0,279.0,289.0,299.0,309.0,319.0,329.0,339.0,349.0,359.0,370.0,380.0,390.0,400.0,410.0
Toys Avenue_1,10.0,0.0,10.0,20.0,30.0,41.0,51.0,61.0,71.0,81.0,91.0,102.0,112.0,122.0,132.0,142.0,152.0,163.0,173.0,183.0,193.0,203.0,213.0,224.0,234.0,244.0,254.0,264.0,274.0,285.0,283.0,282.0,281.0,279.0,278.0,277.0,276.0,275.0,274.0,273.0,...,247.0,254.0,261.0,268.0,275.0,282.0,289.0,296.0,304.0,311.0,318.0,132.0,142.0,152.0,162.0,172.0,182.0,192.0,202.0,212.0,222.0,232.0,242.0,252.0,262.0,273.0,283.0,293.0,303.0,313.0,323.0,333.0,343.0,353.0,363.0,373.0,383.0,393.0,403.0,413.0
Toys Avenue_2,20.0,10.0,0.0,10.0,20.0,30.0,41.0,51.0,61.0,71.0,81.0,91.0,102.0,112.0,122.0,132.0,142.0,152.0,163.0,173.0,183.0,193.0,203.0,213.0,224.0,234.0,244.0,254.0,264.0,274.0,273.0,272.0,270.0,269.0,268.0,267.0,266.0,265.0,264.0,263.0,...,247.0,254.0,261.0,268.0,275.0,283.0,290.0,297.0,304.0,312.0,319.0,136.0,146.0,156.0,166.0,176.0,186.0,196.0,206.0,216.0,226.0,236.0,246.0,256.0,266.0,276.0,286.0,296.0,306.0,316.0,326.0,336.0,346.0,356.0,366.0,376.0,386.0,397.0,407.0,417.0
Toys Avenue_3,30.0,20.0,10.0,0.0,10.0,20.0,30.0,41.0,51.0,61.0,71.0,81.0,91.0,102.0,112.0,122.0,132.0,142.0,152.0,163.0,173.0,183.0,193.0,203.0,213.0,224.0,234.0,244.0,254.0,264.0,263.0,261.0,260.0,259.0,258.0,257.0,256.0,255.0,254.0,253.0,...,247.0,255.0,262.0,269.0,276.0,283.0,291.0,298.0,305.0,313.0,320.0,140.0,150.0,160.0,170.0,180.0,190.0,200.0,210.0,220.0,230.0,240.0,250.0,260.0,270.0,280.0,290.0,300.0,310.0,320.0,330.0,340.0,350.0,360.0,370.0,380.0,390.0,400.0,410.0,420.0
Toys Avenue_4,41.0,30.0,20.0,10.0,0.0,10.0,20.0,30.0,41.0,51.0,61.0,71.0,81.0,91.0,102.0,112.0,122.0,132.0,142.0,152.0,163.0,173.0,183.0,193.0,203.0,213.0,224.0,234.0,244.0,254.0,253.0,251.0,250.0,249.0,248.0,246.0,245.0,244.0,243.0,243.0,...,248.0,255.0,263.0,270.0,277.0,285.0,292.0,299.0,307.0,314.0,321.0,145.0,154.0,164.0,174.0,184.0,194.0,204.0,214.0,224.0,234.0,244.0,254.0,264.0,274.0,284.0,294.0,304.0,314.0,324.0,334.0,344.0,354.0,364.0,374.0,384.0,394.0,404.0,414.0,424.0


### Routing Solver

In [ ]:
# Data Model Creation function
def create_data_model(distance_matrix):
    data = {}
    data['distance_matrix'] = distance_matrix
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data

# Let's create the Data Model with the distance matrix
data = create_data_model(neighboordhood_mtx)

# Routing Model Initialization
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_vehicles'],
                                       data['depot'])

routing = pywrapcp.RoutingModel(manager)

# Distance matrix callback function
def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)

routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Search parameters definition
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
search_parameters.local_search_metaheuristic = (
    routing_enums_pb2.LocalSearchMetaheuristic.AUTOMATIC)

def print_solution(manager, routing, solution):

    route_array = []
    index = routing.Start(0)

    route_distance = 1
    while not routing.IsEnd(index):
        route_array.append(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)

    return route_array

# Solver launch
solution = routing.SolveWithParameters(search_parameters)

if solution:
    route_array = print_solution(manager, routing, solution)


### Route solution and display

In [ ]:
# Let's sort the capital according to the route defined
neighboordhood_augmented_coordinates_sorted = neighboordhood_augmented_coordinates.reset_index().copy()
neighboordhood_augmented_coordinates_sorted["rank"] = np.zeros(len(neighboordhood_augmented_coordinates_sorted))

for index in neighboordhood_augmented_coordinates_sorted.index:
  neighboordhood_augmented_coordinates_sorted.loc[index, "rank"] = route_array.index(index)

neighboordhood_augmented_coordinates_sorted = neighboordhood_augmented_coordinates_sorted.sort_values(by="rank")
print(neighboordhood_augmented_coordinates_sorted)


          Street Name      Long        Lat   rank
0       Toys Avenue_0  3.096106  50.674380    0.0
204  Snowman Street_0  3.096121  50.674380    1.0
205  Snowman Street_1  3.096168  50.674391    2.0
206  Snowman Street_2  3.096215  50.674402    3.0
207  Snowman Street_3  3.096262  50.674413    4.0
..                ...       ...        ...    ...
5       Toys Avenue_5  3.095735  50.674727  286.0
4       Toys Avenue_4  3.095809  50.674658  287.0
3       Toys Avenue_3  3.095883  50.674588  288.0
2       Toys Avenue_2  3.095958  50.674519  289.0
1       Toys Avenue_1  3.096032  50.674449  290.0

[291 rows x 4 columns]


In [ ]:
fig = go.Figure(data=go.Scatter(x=neighboordhood_augmented_coordinates_sorted["Long"], y=neighboordhood_augmented_coordinates_sorted["Lat"]))
fig = fig.update_layout(paper_bgcolor='rgba(0, 0, 0, 0)',
                        plot_bgcolor='rgba(0, 0, 0, 0)',
                        xaxis=dict(showline=False, showgrid=False),
                        yaxis=dict(showline=False, showgrid=False))
fig = fig.update_xaxes(showticklabels=False)
fig = fig.update_yaxes(showticklabels=False, scaleanchor="x", scaleratio=2)

fig.show()


In [ ]:
# Let's add this journey to the World Map (credit to https://github.com/matthiashhh)
folium.PolyLine(neighboordhood_augmented_coordinates_sorted[["Lat", "Long"]].to_numpy(), color='blue').add_to(neighboordhood_map)
neighboordhood_map